<a href="https://colab.research.google.com/github/p4r1ch4y/clirnet_assignment/blob/main/dspy_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DSPy Assignment


Time  : 5/11/25 5:30pm - 6/11/25 2:30 am

Subrata Choudhury - Backend Engineer - Round 1

In [20]:
!pip install dspy-ai requests beautifulsoup4 pandas lxml


In [21]:
import json
import dspy
import copy
from typing import List, Optional
from typing import Literal, Dict, Union
from dspy.adapters import XMLAdapter
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Add API Keys

In [22]:
API_KEY="Try_ur__API_get_api_from_longcat.chat"
main_lm = dspy.LM("openai/LongCat-Flash-Chat", api_key=API_KEY, api_base="https://api.longcat.chat/openai/v1")

dspy.settings.configure(lm=main_lm, adapter=dspy.XMLAdapter())
print("DSPy set up! Hope the key works.")


DSPy set up! Hope the key works.


In [23]:
# urls to scrap from the problem doc
urls = [

"https://en.wikipedia.org/wiki/Sustainable_agriculture ",
"https://www.nature.com/articles/d41586-025-03353-5 ",
"https://www.sciencedirect.com/science/article/pii/S1043661820315152 ",
"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/ ",
"https://www.fao.org/3/y4671e/y4671e06.htm ",
"https://www.medscape.com/viewarticle/time-reconsider-tramadol-chronic-pain-2025a1000ria ",
"https://www.sciencedirect.com/science/article/pii/S0378378220307088 ",
"https://www.frontiersin.org/news/2025/09/01/rectangle-telescope-finding-habitable-planets ",
"https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7 ",
"https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india ",

]
print(f"Got {len(urls)} URLs ready to scrape and process.")


Got 10 URLs ready to scrape and process.


dummy user agent to avoid block

In [24]:
def get_text_from_url(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
        response = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')

        for tag in soup(['script', 'style', 'nav', 'header', 'footer']):
            tag.decompose()

        paragraphs = soup.find_all('p')
        text = ' '.join([p.get_text() for p in paragraphs])

        text = ' '.join(text.split())
        if len(text) > 4000:
            text = text[:4000] + " ... (text cut off to fit)"

        print(f"Got {len(text)} chars from {url.split('/')[2]}")
        return text
    except Exception as e:
        print(f"Scrape failed for {url}: {e}")
        return ""

test_url = urls[0]
test_text = get_text_from_url(test_url)
if test_text:
    print("Sample text start:", test_text[:100])

Got 4026 chars from en.wikipedia.org
Sample text start: Sustainable agriculture is farming in sustainable ways meeting society's present food and textile ne


In [25]:
entity extractor`

SyntaxError: invalid syntax (ipython-input-4000180668.py, line 1)

In [26]:
from typing import List, Dict, Tuple
from pydantic import BaseModel, Field

class EntityWithAttr(BaseModel):
    entity: str = Field(description="the named entity")
    attr_type: str = Field(description="semantic type of the entity (e.g. Drug, Disease, Symptom, etc.)")

class ExtractEntities(dspy.Signature):
    """From the paragraph extract all relevant entities and their semantic attribute types."""
    paragraph: str = dspy.InputField(desc="input paragraph")
    entities: List[EntityWithAttr] = dspy.OutputField(desc="list of entities and their attribute types")

extractor = dspy.Predict(ExtractEntities)
print("Entity extractor copied from sample.")

Entity extractor copied from sample.


deduplication

In [27]:
# ---------------------------------------------------------
# 2.  DEDUPLICATOR (recursive batching + confidence loop)
# ---------------------------------------------------------
class DeduplicateEntities(dspy.Signature):
    """Given a list of (entity, attr_type) decide which ones are duplicates.
    Return a deduplicated list and a confidence that the remaining items are ALL distinct."""
    items: List[EntityWithAttr] = dspy.InputField(desc="batch of entities to deduplicate")
    deduplicated: List[EntityWithAttr] = dspy.OutputField(desc="deduplicated list")
    confidence: float = dspy.OutputField(
        desc="confidence (0-1) that every item in deduplicated is semantically distinct"
    )

dedup_predictor = dspy.ChainOfThought(DeduplicateEntities)

def deduplicate_with_lm(
    items: List[EntityWithAttr],
    *,
    batch_size: int = 10,
    target_confidence: float = 0.9,
) -> List[EntityWithAttr]:
    """
    Recursively deduplicate using the LM.
    Works by:
    1. splitting into batches of `batch_size`
    2. for each batch asking the LM for duplicates + confidence
    3. rerunning the batch until confidence >= target_confidence
    4. concatenating results from all batches
    """
    if not items:
        return []

    # helper to process one batch
    def _process_batch(batch: List[EntityWithAttr]) -> List[EntityWithAttr]:
        while True:
            pred = dedup_predictor(items=batch)
            if pred.confidence >= target_confidence:
                return pred.deduplicated
            # otherwise loop again with same batch

    # split into batches and process
    results = []
    for i in range(0, len(items), batch_size):
        batch = items[i : i + batch_size]
        results.extend(_process_batch(batch))
    return results

print("Deduplication function from sample ready. Uses loops!")

Deduplication function from sample ready. Uses loops!


In [28]:
# ---------------------------------------------------------
#  RELATION EXTRACTION
# ---------------------------------------------------------

class Relation(BaseModel):
    subj: str = Field(description="subject entity (exact string as in deduplicated list)")
    pred: str = Field(description="short predicate / relation phrase")
    obj: str = Field(description="object entity (exact string as in deduplicated list)")

class ExtractRelations(dspy.Signature):
    """Given the original paragraph and a list of unique entities, extract all factual (subject, predicate, object) triples that are explicitly stated or clearly implied."""
    paragraph: str = dspy.InputField(desc="original paragraph")
    entities: List[str] = dspy.InputField(desc="list of deduplicated entity strings")
    relations: List[Relation] = dspy.OutputField(desc="list of subject-predicate-object triples")

rel_predictor = dspy.ChainOfThought(ExtractRelations)
print("Relation extractor from sample.")

Relation extractor from sample.


In [30]:
# ---------------------------------------------------------
#   MERMAID SERIALISER  (revised)
# ---------------------------------------------------------

def triples_to_mermaid(
    triples: list[Relation],
    entity_list: list[str],
    max_label_len: int = 40
) -> str:
    """
    Convert triples to a VALID Mermaid flowchart LR diagram.
    """
    entity_set = {e.strip().lower() for e in entity_list}
    lines = ["flowchart LR"]

    def _make_id(s: str) -> str:
      # Create valid Mermaid node ID (no spaces or special chars)
        return s.strip().replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_")

    for t in triples:
        subj_norm, obj_norm = t.subj.strip().lower(), t.obj.strip().lower()

        if obj_norm in entity_set:
            src, dst, lbl = t.subj, t.obj, t.pred
        elif subj_norm in entity_set:
            src, dst, lbl = t.obj, t.subj, t.pred
        else:
            continue

        # Sanitize label
        lbl = lbl.strip()
        if len(lbl) > max_label_len:
            lbl = lbl[:max_label_len - 3] + "..."

        # Use valid IDs with display labels
        src_id, dst_id = _make_id(src), _make_id(dst)
        lines.append(f' {src_id}["{src}"] -->|{lbl}| {dst_id}["{dst}"]')

    return "\n".join(lines)

print("Mermaid function from sample. Makes graphs!")


Mermaid function from sample. Makes graphs!


main funtion

In [31]:
def process_url(url, index):
    print(f"\n--- Doing URL {index+1}/10: {url} ---")

    paragraph = get_text_from_url(url)
    if not paragraph:
        return {'url': url, 'entities': [], 'relations': [], 'mermaid': 'No text'}

    time.sleep(2)

    extracted = extractor(paragraph=paragraph)
    print(f"Found {len(extracted.entities)} raw entities")

    unique = deduplicate_with_lm(extracted.entities, batch_size=10, target_confidence=0.9)
    entity_strings = [e.entity for e in unique]
    print(f"After dedup: {len(unique)} unique")

    rel_out = rel_predictor(paragraph=paragraph, entities=entity_strings)
    print(f"Found {len(rel_out.relations)} relations")

    mermaid_code = triples_to_mermaid(rel_out.relations, entity_strings)

    filename = f'mermaid_{index+1:02d}.md'
    with open(filename, 'w') as f:
        f.write(mermaid_code)
    print(f"Saved {filename}")

    return {
        'url': url,
        'entities': unique,
        'relations': rel_out.relations,
        'mermaid': mermaid_code
    }

print("My process function ready. Uses sample code inside.")


My process function ready. Uses sample code inside.


**Run on all URLS with loop**

In [32]:
all_results = []
for i, url in enumerate(urls):
    try:
        result = process_url(url, i)
        all_results.append(result)
    except Exception as e:
        print(f"Error on URL {i+1}: {e}")
        all_results.append({'url': url, 'entities': [], 'relations': [], 'mermaid': 'Error'})

print("\nAll done! Processed 10 URLs.")



--- Doing URL 1/10: https://en.wikipedia.org/wiki/Sustainable_agriculture  ---
Got 4026 chars from en.wikipedia.org
Found 46 raw entities
After dedup: 46 unique
Found 50 relations
Saved mermaid_01.md

--- Doing URL 2/10: https://www.nature.com/articles/d41586-025-03353-5  ---
Got 453 chars from www.nature.com
Found 0 raw entities
After dedup: 0 unique
Found 0 relations
Saved mermaid_02.md

--- Doing URL 3/10: https://www.sciencedirect.com/science/article/pii/S1043661820315152  ---
Got 0 chars from www.sciencedirect.com

--- Doing URL 4/10: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/  ---
Got 373 chars from www.ncbi.nlm.nih.gov
Found 8 raw entities
After dedup: 7 unique
Found 8 relations
Saved mermaid_04.md

--- Doing URL 5/10: https://www.fao.org/3/y4671e/y4671e06.htm  ---
Got 50 chars from www.fao.org
Found 0 raw entities
After dedup: 0 unique
Found 0 relations
Saved mermaid_05.md

--- Doing URL 6/10: https://www.medscape.com/viewarticle/time-reconsider-tramadol-chronic-pa

In [33]:
csv_rows = []
for result in all_results:
    url = result['url']
    for ent in result['entities']:
        csv_rows.append({
            'link': url,
            'tag': ent.entity,
            'tag_type': ent.attr_type
        })

df = pd.DataFrame(csv_rows)
if not df.empty:
    df = df.drop_duplicates()
    df.to_csv('tags.csv', index=False)
    print(f"Saved tags.csv: {len(df)} rows total")
    print(df.head(10))
else:
    print("No entities for CSV :(")

if not df.empty:
    print("\nTop types:")
    print(df['tag_type'].value_counts().head(5))

Saved tags.csv: 52 rows total
                                                link  \
0  https://en.wikipedia.org/wiki/Sustainable_agri...   
1  https://en.wikipedia.org/wiki/Sustainable_agri...   
2  https://en.wikipedia.org/wiki/Sustainable_agri...   
3  https://en.wikipedia.org/wiki/Sustainable_agri...   
4  https://en.wikipedia.org/wiki/Sustainable_agri...   
5  https://en.wikipedia.org/wiki/Sustainable_agri...   
6  https://en.wikipedia.org/wiki/Sustainable_agri...   
7  https://en.wikipedia.org/wiki/Sustainable_agri...   
8  https://en.wikipedia.org/wiki/Sustainable_agri...   
9  https://en.wikipedia.org/wiki/Sustainable_agri...   

                           tag             tag_type  
0      Sustainable agriculture              Concept  
1                      farming             Activity  
2                         food             Resource  
3                textile needs             Resource  
4           ecosystem services              Concept  
5               sustainabilit

summary

In [34]:
print("="*50)
print("ASSIGNMENT COMPLETE!")
print("="*50)
total_entities = sum(len(r['entities']) for r in all_results)
total_rels = sum(len(r['relations']) for r in all_results)
print(f"URLs: {len(urls)}")
print(f"Total entities (after dedup): {total_entities}")
print(f"Total relations: {total_rels}")
print("Mermaid files: mermaid_01.md to mermaid_10.md")
print("CSV: tags.csv with link, tag, tag_type")
print("Paste Mermaids into mermaid.live to see graphs.")
print("I copied most from sample as i'm very new, added scrape and loop. Hope it works ")


ASSIGNMENT COMPLETE!
URLs: 10
Total entities (after dedup): 53
Total relations: 58
Mermaid files: mermaid_01.md to mermaid_10.md
CSV: tags.csv with link, tag, tag_type
Paste Mermaids into mermaid.live to see graphs.
I copied most from sample as i'm very new, added scrape and loop. Hope it works 
